<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_prompt_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpt-prompt-engineer
Автор: Matt Shumer (https://twitter.com/mattshumer_)

Репозиторий: https://github.com/mshumer/gpt-prompt-engineer

Инструмент подбирает оптимальный промпт для заданной задачи.

Как пользоваться:
1. Создайте файл _secrets.py из _secrets.example.py и укажите в нём OpenAI API ключ.
2. Если нет доступа к GPT-4, в ячейке с конфигом замените `CANDIDATE_MODEL = 'gpt-4'` на `CANDIDATE_MODEL = 'gpt-3.5-turbo'`.
3. В последней ячейке заполните описание задачи, до 15 тестовых случаев и количество генерируемых промптов.
4. Запустите все ячейки. ИИ сгенерирует варианты промптов и по ELO выберет лучшие.

In [12]:
%pip install openai==0.28 prettytable tqdm tenacity wandb -qq

Note: you may need to restart the kernel to use updated packages.


In [23]:
from prettytable import PrettyTable
import time
import openai
from tqdm import tqdm
import itertools
import wandb
from tenacity import retry, stop_after_attempt, wait_exponential

try:
    from _secrets import OPENAI_API_KEY
except ImportError:
    OPENAI_API_KEY = ""  # создайте _secrets.py из _secrets.example.py и укажите ключ
openai.api_key = OPENAI_API_KEY

use_wandb = False # True — логировать конфиг и результаты в Weights & Biases

use_portkey = False # True — логировать цепочки промптов и ответы в Portkey (https://portkey.ai/)

In [24]:
system_gen_system_prompt = """Твоя задача — генерировать системные промпты для GPT-4 по описанию сценария использования и тестовым случаям.

Промпты должны быть для произвольных задач: заголовок лендинга, вводный абзац, решение задачи по математике и т.п.

В сгенерированном промпте опиши на русском языке, как должна вести себя модель: что она получает на вход и что может выводить. Будь креативен, чтобы получить лучший результат. Напоминать модели, что она ИИ, не нужно.

Оценка будет по качеству работы промпта — не жульничай: не включай в промпт детали из тестовых случаев. Промпты с примерами из тестов дисквалифицируются.

Главное: в ответе должен быть только текст промпта, без пояснений и лишнего текста."""


ranking_system_prompt = """Твоя задача — оценить качество двух ответов, сгенерированных разными промптами для одной и той же задачи.

Тебе даны: описание задачи, тестовый запрос и два варианта ответа (A и B).

Выбери лучший по качеству. Если лучше вариант A — ответь одной буквой 'A'. Если лучше вариант B — ответь одной буквой 'B'.

«Лучше» значит заметно лучше, а не чуть-чуть. Будь строгим критиком: помечай как лучший только тот вариант, который действительно впечатляет больше.

В ответе — только одна буква: A или B. Оценивай беспристрастно."""

In [15]:
# Коэффициент K для изменения рейтинга ELO
K = 32

CANDIDATE_MODEL = 'gpt-4o-mini'  # или gpt-4.1-mini, gpt-4o — см. доступ в OpenAI
CANDIDATE_MODEL_TEMPERATURE = 0.9

GENERATION_MODEL = 'gpt-4o-mini'
GENERATION_MODEL_TEMPERATURE = 0.8
GENERATION_MODEL_MAX_TOKENS = 60

N_RETRIES = 3  # сколько раз повторять запрос к модели ранжирования при ошибке
RANKING_MODEL = 'gpt-4o-mini'
RANKING_MODEL_TEMPERATURE = 0.5

NUMBER_OF_PROMPTS = 10 # сколько вариантов промптов генерировать (больше — дороже, но лучше результат)

WANDB_PROJECT_NAME = "gpt-prompt-eng" # имя проекта в Weights & Biases при use_wandb = True
WANDB_RUN_NAME = None # имя запуска в W&B для идентификации (опционально)

PORTKEY_API = "" # API ключ Portkey при use_portkey = True: https://app.portkey.ai/
PORTKEY_TRACE = "prompt_engineer_test_run" # ID трассы для отличия цепочек промптов
HEADERS = {} # не менять — заполняется при use_portkey = True

In [16]:
def start_wandb_run():
  # запуск сессии wandb и сохранение конфига
  wandb.init(
    project=WANDB_PROJECT_NAME, 
    name=WANDB_RUN_NAME,
    config={
      "K": K,
      "system_gen_system_prompt": system_gen_system_prompt, 
      "ranking_system_prompt": ranking_system_prompt,
      "candidate_model": CANDIDATE_MODEL,
      "candidate_model_temperature": CANDIDATE_MODEL_TEMPERATURE,
      "generation_model": GENERATION_MODEL,
      "generation_model_temperature": GENERATION_MODEL_TEMPERATURE,
      "generation_model_max_tokens": GENERATION_MODEL_MAX_TOKENS,
      "n_retries": N_RETRIES,
      "ranking_model": RANKING_MODEL,
      "ranking_model_temperature": RANKING_MODEL_TEMPERATURE,
      "number_of_prompts": NUMBER_OF_PROMPTS
      })
  
  return 

In [17]:
# По желанию: логирование конфига, промптов и результатов в Weights & Biases
if use_wandb:
  start_wandb_run()

In [27]:
def start_portkey_run():
  # заголовки Portkey для логирования промптов и ответов
  openai.api_base="https://api.portkey.ai/v1/proxy"
  HEADERS = {
    "x-portkey-api-key": PORTKEY_API, 
    "x-portkey-mode": "proxy openai",
    "x-portkey-trace-id": PORTKEY_TRACE,
    #"x-portkey-retry-count": 5 # авто-повторы с экспоненциальной задержкой при сбое OpenAI
  } 
  return HEADERS

In [19]:
# По желанию: логирование промптов и ответов в Portkey
if use_portkey:
    HEADERS=start_portkey_run()

In [20]:
def generate_candidate_prompts(description, test_cases, number_of_prompts):
  outputs = openai.ChatCompletion.create(
      model=CANDIDATE_MODEL, # при отсутствии GPT-4 замените на 'gpt-3.5-turbo'
      messages=[
          {"role": "system", "content": system_gen_system_prompt},
          {"role": "user", "content": f"Вот тестовые случаи: `{test_cases}`\n\nОписание сценария: `{description.strip()}`\n\nОтветь только текстом промпта, без пояснений. Будь креативен."}
          ],
      temperature=CANDIDATE_MODEL_TEMPERATURE,
      n=number_of_prompts,
      headers=HEADERS)

  prompts = []

  for i in outputs.choices:
    prompts.append(i.message.content)
  return prompts

def expected_score(r1, r2):
    return 1 / (1 + 10**((r2 - r1) / 400))

def update_elo(r1, r2, score1):
    e1 = expected_score(r1, r2)
    e2 = expected_score(r2, r1)
    return r1 + K * (score1 - e1), r2 + K * ((1 - score1) - e2)

# Ранжирование: до N_RETRIES повторов с экспоненциальной задержкой.
@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_score(description, test_case, pos1, pos2, ranking_model_name, ranking_model_temperature):    
    score = openai.ChatCompletion.create(
        model=ranking_model_name,
        messages=[
            {"role": "system", "content": ranking_system_prompt},
            {"role": "user", "content": f"""Задача: {description.strip()}
Запрос: {test_case['prompt']}
Вариант A: {pos1}
Вариант B: {pos2}"""}
        ],
        logit_bias={
              '32': 100,  # токен 'A'
              '33': 100,  # токен 'B'
        },
        max_tokens=1,
        temperature=ranking_model_temperature,
        headers=HEADERS,
    ).choices[0].message.content
    return score

@retry(stop=stop_after_attempt(N_RETRIES), wait=wait_exponential(multiplier=1, min=4, max=70))
def get_generation(prompt, test_case):
    generation = openai.ChatCompletion.create(
        model=GENERATION_MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": f"{test_case['prompt']}"}
        ],
        max_tokens=GENERATION_MODEL_MAX_TOKENS,
        temperature=GENERATION_MODEL_TEMPERATURE,
        headers=HEADERS,
    ).choices[0].message.content
    return generation

def test_candidate_prompts(test_cases, description, prompts):
  # Начальный рейтинг ELO для каждого промпта — 1200
  prompt_ratings = {prompt: 1200 for prompt in prompts}

  # Число раундов для прогресс-бара
  total_rounds = len(test_cases) * len(prompts) * (len(prompts) - 1) // 2

  pbar = tqdm(total=total_rounds, ncols=70)

  # Для каждой пары промптов
  for prompt1, prompt2 in itertools.combinations(prompts, 2):
      for test_case in test_cases:
          pbar.update()

          generation1 = get_generation(prompt1, test_case)
          generation2 = get_generation(prompt2, test_case)

          # Ранжируем ответы
          score1 = get_score(description, test_case, generation1, generation2, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)
          score2 = get_score(description, test_case, generation2, generation1, RANKING_MODEL, RANKING_MODEL_TEMPERATURE)

          score1 = 1 if score1 == 'A' else 0 if score1 == 'B' else 0.5
          score2 = 1 if score2 == 'B' else 0 if score2 == 'A' else 0.5

          score = (score1 + score2) / 2

          r1, r2 = prompt_ratings[prompt1], prompt_ratings[prompt2]
          r1, r2 = update_elo(r1, r2, score)
          prompt_ratings[prompt1], prompt_ratings[prompt2] = r1, r2

          if score > 0.5:
              print(f"Победитель: {prompt1}")
          elif score < 0.5:
              print(f"Победитель: {prompt2}")
          else:
              print("Ничья")

  pbar.close()

  return prompt_ratings


def generate_optimal_prompt(description, test_cases, number_of_prompts=10, use_wandb=False): 
  if use_wandb:
    wandb_table = wandb.Table(columns=["Промпт", "Рейтинг"])
    if wandb.run is None:
      start_wandb_run()

  prompts = generate_candidate_prompts(description, test_cases, number_of_prompts)
  prompt_ratings = test_candidate_prompts(test_cases, description, prompts)

  # Итоговая таблица рейтингов ELO
  table = PrettyTable()
  table.field_names = ["Промпт", "Рейтинг"]
  for prompt, rating in sorted(prompt_ratings.items(), key=lambda item: item[1], reverse=True):
      table.add_row([prompt, rating])
      if use_wandb:
         wandb_table.add_data(prompt, rating)

  if use_wandb: # сохранить результаты в таблицу Weights & Biases и завершить сессию
    wandb.log({"prompt_ratings": wandb_table})
    wandb.finish()
  print(table)

# В ячейке ниже укажите описание задачи и тестовые случаи

In [21]:
description = "По запросу сгенерировать заголовок для лендинга." # такое описание обычно даёт хороший результат

test_cases = [
    {'prompt': 'Продвижение нового фитнес-приложения Smartly'},
    {'prompt': 'Почему веганская диета полезна для здоровья'},
    {'prompt': 'Запуск онлайн-курса по цифровому маркетингу'},
    {'prompt': 'Запуск линейки экологичной одежды'},
    {'prompt': 'Продвижение блога о бюджетных путешествиях'},
    {'prompt': 'Реклама ПО для управления проектами'},
    {'prompt': 'Презентация книги по изучению Python'},
    {'prompt': 'Продвижение платформы для изучения языков'},
    {'prompt': 'Реклама сервиса персональных планов питания'},
    {'prompt': 'Запуск приложения для ментального здоровья и медитации'},
]

if use_wandb:
    wandb.config.update({"description": description, 
                        "test_cases": test_cases})

In [28]:
generate_optimal_prompt(description, test_cases, NUMBER_OF_PROMPTS, use_wandb)

PermissionError: Project `proj_xKWNFYgcA7UQIAIRSlXZ6ey3` does not have access to model `gpt-4.1-mini`